In [1]:
import re

button_regex = re.compile(r"Button (?P<button>[AB]): X\+(?P<X>\d+), Y\+(?P<Y>\d+)", re.IGNORECASE)
prize_regex = re.compile(r"Prize: X=(?P<X>\d+), Y=(?P<Y>\d+)", re.IGNORECASE)

machines = []
with open("Day13.txt") as file:
    machine = []
    for line in file:
        if match := button_regex.match(line):
            b, x, y = match.groups()
            machine.extend([int(x), int(y)])
        elif match := prize_regex.match(line):
            x, y = match.groups()
            machine.extend([int(x), int(y)])
        else:
            machines.append(machine)
            machine = []
    machines.append(machine)

### Numpy (fastest)

In [2]:
import numpy as np

def solve(
    button_a_x, button_a_y, 
    button_b_x, button_b_y, 
    prize_x, prize_y, offset=0,
    cost_a=3, cost_b=1,
):
    prize_x, prize_y = prize_x + offset, prize_y + offset
    solution = np.linalg.solve(
        np.array([
            [button_a_x, button_b_x],
            [button_a_y, button_b_y],
        ], np.int64),
        np.array([
            [prize_x],
            [prize_y],
        ], np.int64),
    )
    press_a, press_b = round(solution[0][0]), round(solution[1][0])
    if (
        press_a * button_a_x + press_b * button_b_x, 
        press_a * button_a_y + press_b * button_b_y,
    ) == (prize_x, prize_y):
        return press_a * cost_a, press_b * cost_b
    return 0, 0

In [3]:
%%time
sum(sum(solve(*machine)) for machine in machines)

CPU times: user 127 ms, sys: 46 μs, total: 127 ms
Wall time: 4.43 ms


35255

In [4]:
%%time
sum(sum(solve(*machine, offset=10_000_000_000_000)) for machine in machines)

CPU times: user 125 ms, sys: 14 μs, total: 125 ms
Wall time: 4.57 ms


87582154060429

### Z3 Solver (slowest)

In [5]:
import z3

def solve_z3(
    button_a_x, button_a_y, 
    button_b_x, button_b_y, 
    prize_x, prize_y, offset=0,
    cost_a=3, cost_b=1,
):
    solver = z3.Solver()
    press_a, press_b = z3.Ints("press_a press_b")
    solver.add(press_a > 0)
    solver.add(press_b > 0)
    if not offset:
        solver.add(press_a <= 100)
        solver.add(press_b <= 100)
    solver.add(press_a * button_a_x + press_b * button_b_x == prize_x + offset)
    solver.add(press_a * button_a_y + press_b * button_b_y == prize_y + offset)
    if solver.check() != z3.sat:
        return 0, 0
    solution = solver.model()
    press_a, press_b = solution[press_a].as_long(), solution[press_b].as_long()
    return press_a * cost_a, press_b * cost_b

In [6]:
%%time
sum(sum(solve_z3(*machine)) for machine in machines)

CPU times: user 1.54 s, sys: 10.1 ms, total: 1.55 s
Wall time: 416 ms


35255

In [7]:
%%time
sum(sum(solve_z3(*machine, offset=10_000_000_000_000)) for machine in machines)

CPU times: user 372 ms, sys: 6.94 ms, total: 378 ms
Wall time: 376 ms


87582154060429

### Sympy

In [8]:
import sympy

def solve_sympy(
    button_a_x, button_a_y, 
    button_b_x, button_b_y, 
    prize_x, prize_y, offset=0,
    cost_a=3, cost_b=1,
):
    prize_x, prize_y = prize_x + offset, prize_y + offset
    A = sympy.Matrix([
        [button_a_x, button_b_x],
        [button_a_y, button_b_y],
    ])
    b = sympy.Matrix([
        [prize_x],
        [prize_y],
    ])
    press_a, press_b = sympy.symbols("press_a, press_b")
    solution = sympy.linsolve((A, b), [press_a, press_b])
    press_a, press_b = [int(x) for sol in solution for x in sol]
    if (
        press_a * button_a_x + press_b * button_b_x, 
        press_a * button_a_y + press_b * button_b_y,
    ) == (prize_x, prize_y):
        return press_a * cost_a, press_b * cost_b
    return 0, 0

In [9]:
%%time
sum(sum(solve_sympy(*machine)) for machine in machines)

CPU times: user 217 ms, sys: 3.95 ms, total: 221 ms
Wall time: 221 ms


35255

In [10]:
%%time
sum(sum(solve_sympy(*machine, offset=10_000_000_000_000)) for machine in machines)

CPU times: user 191 ms, sys: 1 ms, total: 192 ms
Wall time: 192 ms


87582154060429